In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np      # linear algebra
import pandas as pd     # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        cars_path = os.path.join(dirname, filename)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#create datafram from csv
car_df = pd.read_csv(cars_path)

#drop string columns
car_df.drop(axis=1,labels=['SellerName','StreetName','State','Zipcode','VIN','Stock#','Make',
                           'Model','Used/New','SellerType','DealType','ExteriorColor',
                           'InteriorColor', 'Drivetrain','FuelType','Transmission','Engine'],inplace=True)

keys = list(car_df.columns)
keys.remove('Price')
#convert 'Price' column into integers
for i in range(len(car_df)):
    
    #replace each value in 'Price' with the integer value of the price
    price_digits = ''
    for c in car_df.loc[i,'Price']:
        if c.isdigit():
            price_digits+=c
    if len(price_digits) == 0:
        price_digits = float('nan')
    else:
        car_df.loc[i,'Price'] = int(price_digits)

#convert dataframe to numpy 2D array
car_mat = car_df.to_numpy()

#find average price
no_price = 0
total = 0
for car in car_mat:
    if(car[1] == 'Not Priced'):
        no_price+=1
    else:
        total+=car[1]
avg_price = total/(len(car_mat)-no_price)

#replace unpriced cars with average value
for car in car_mat:
    if(car[1] == 'Not Priced'):
        car[1] = avg_price

#separate price column from matrix into a new array
prices = car_mat[:,1]
car_mat = np.delete(car_mat,1,axis=1)        

#perform linear regression between prices and each column in car_mat
factors = [] #to store each trait and its effect on price
for i in range(car_mat.shape[1]-1):
    col = car_mat[:,i+1]
    col = col.tolist()
    
    #regression and finding r^2 value
    res = stats.linregress(col,prices.tolist())
    r_squared = res.rvalue**2
    
    #add trait and corresponding r^2 value to list
    factors.append([keys[i],r_squared])

factors.sort(key=lambda x:x[1],reverse=True)
print('Most influential factors on price:')
for i in range(len(factors)):
    print('#','{:2}'.format(i+1),':\t','{:20}'.format(factors[i][0]),'\t','{:.4f}'.format(factors[i][1]))